In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from sklearn.manifold.t_sne import _joint_probabilities
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)

ModuleNotFoundError: ignored

In [6]:
import pandas as pd
data=pd.read_csv('transposed.csv')
#pd.set_option('display.max_columns',None)
data.head()

,gene,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2ML1|144568,A2M|2,A4GALT|53947,A4GNT|51146,AAA1|404744,...,SLC6A3|6531,SLC6A4|6532,SLC6A5|9152,SLC6A6|6533,SLC6A7|6534,SLC6A8|6535,SLC6A9|6536,SLC7A10|56301,SLC7A11|23657,SLC7A13|157724
0,TCGA-2A-A8VL-01A-21R-A37L-07,67.9197,0.0000,25.2317,136.0453,45.3141,4827.8527,767.7652,0.5149,0.0,...,3.6045,2.0597,0.0,133.8826,3.6045,1382.5953,73.9289,1.5808,124.0989,2.0597
1,TCGA-2A-A8VO-01A-11R-A37L-07,17.9448,0.0000,17.9448,169.1662,2.6585,20000.6912,503.7834,1.3292,0.0,...,17.2802,1.3292,0.0,145.5522,0.6646,1357.1578,43.5128,1.3758,93.7117,1.9939
2,TCGA-2A-A8VT-01A-11R-A37L-07,17.1029,0.3676,1.8382,159.0037,1.1029,4906.4963,251.1029,0.3676,0.0,...,2.9412,5.5147,0.0,188.9706,1.1029,1845.9559,68.6287,2.9669,124.2647,0.0000
3,TCGA-2A-A8VV-01A-11R-A37L-07,17.6115,0.0000,3.7651,175.2959,0.9413,8427.4903,464.5252,0.4706,0.0,...,3.7651,0.4706,0.0,242.3815,0.4706,1487.7044,131.3002,1.9014,268.7375,0.9413
4,TCGA-2A-A8VX-01A-11R-A37L-07,43.5168,0.0000,0.5038,263.1062,2.5192,8330.8225,458.4960,0.0000,0.0,...,86.1569,2.0154,0.0,405.0888,1.5115,1852.6263,137.1155,1.0430,331.5279,13.6037


In [7]:
data.describe()

,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2ML1|144568,A2M|2,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,...,SLC6A3|6531,SLC6A4|6532,SLC6A5|9152,SLC6A6|6533,SLC6A7|6534,SLC6A8|6535,SLC6A9|6536,SLC7A10|56301,SLC7A11|23657,SLC7A13|157724
count,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,...,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000
mean,64.773957,3.590567,15.410773,160.365759,35.546997,13727.783731,583.042428,0.337941,0.008565,859.881484,...,15.545567,8.125066,0.010714,317.556055,1.097827,1754.379172,115.424162,1.700273,262.355746,3.200598
std,76.848046,18.315702,33.512141,70.512812,81.104631,12234.222580,380.550035,0.560213,0.065604,193.796880,...,34.414235,57.188283,0.074295,208.389904,1.552853,623.108994,57.088018,4.116536,265.120523,5.111657
min,2.164500,0.000000,0.000000,15.044100,0.000000,1511.217100,53.936300,0.000000,0.000000,412.081300,...,0.000000,0.000000,0.000000,14.708200,0.000000,695.024100,19.793100,0.000000,10.777500,0.000000
25%,22.327125,0.000000,2.352875,106.437675,5.365725,6502.051775,346.828700,0.000000,0.000000,740.295700,...,1.627275,0.849475,0.000000,177.167300,0.000000,1358.648025,77.947925,0.433175,107.109750,0.466000
50%,41.160650,0.000000,7.218650,147.767300,14.687300,10426.711600,507.692100,0.000000,0.000000,841.497450,...,4.336650,1.743600,0.000000,277.756450,0.652900,1636.133100,105.484850,0.961950,197.978750,1.564250
75%,74.030250,0.433400,15.760575,199.717900,35.152750,17206.441225,709.702750,0.501625,0.000000,959.865475,...,12.035100,4.850525,0.000000,414.557400,1.435575,1955.593625,136.963075,1.757225,329.851300,3.792550
max,729.903900,237.832900,533.582100,476.299200,1276.867000,113991.666100,3474.556200,5.672600,0.745200,1710.851000,...,259.292300,1293.891500,0.993500,2624.079700,16.260200,7167.721200,488.621800,82.279100,3354.010100,45.803000


In [8]:
dat=data.iloc[:,1:]
dat.head()

,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2ML1|144568,A2M|2,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,...,SLC6A3|6531,SLC6A4|6532,SLC6A5|9152,SLC6A6|6533,SLC6A7|6534,SLC6A8|6535,SLC6A9|6536,SLC7A10|56301,SLC7A11|23657,SLC7A13|157724
0,67.9197,0.0000,25.2317,136.0453,45.3141,4827.8527,767.7652,0.5149,0.0,922.7600,...,3.6045,2.0597,0.0,133.8826,3.6045,1382.5953,73.9289,1.5808,124.0989,2.0597
1,17.9448,0.0000,17.9448,169.1662,2.6585,20000.6912,503.7834,1.3292,0.0,921.1659,...,17.2802,1.3292,0.0,145.5522,0.6646,1357.1578,43.5128,1.3758,93.7117,1.9939
2,17.1029,0.3676,1.8382,159.0037,1.1029,4906.4963,251.1029,0.3676,0.0,801.4706,...,2.9412,5.5147,0.0,188.9706,1.1029,1845.9559,68.6287,2.9669,124.2647,0.0000
3,17.6115,0.0000,3.7651,175.2959,0.9413,8427.4903,464.5252,0.4706,0.0,878.6916,...,3.7651,0.4706,0.0,242.3815,0.4706,1487.7044,131.3002,1.9014,268.7375,0.9413
4,43.5168,0.0000,0.5038,263.1062,2.5192,8330.8225,458.4960,0.0000,0.0,1017.2566,...,86.1569,2.0154,0.0,405.0888,1.5115,1852.6263,137.1155,1.0430,331.5279,13.6037


In [9]:
from sklearn import preprocessing

x = dat.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dat = pd.DataFrame(x_scaled)

In [ ]:
print(dat.head)

            0         1         2         3         4         5         6    \
0      0.000116  0.000029  0.000143  0.000029  0.000121  0.000010  0.000112   
1      0.000000  0.000000  0.000003  0.000000  0.000000  0.000000  0.000000   
2      0.000043  0.000029  0.000015  0.000006  0.000001  0.000000  0.000010   
3      0.000232  0.000269  0.001333  0.000290  0.000730  0.000269  0.000794   
4      0.000077  0.000004  0.000009  0.000002  0.000007  0.000006  0.000008   
...         ...       ...       ...       ...       ...       ...       ...   
18772  0.000483  0.000420  0.001110  0.000667  0.000565  0.000020  0.001032   
18773  0.003186  0.002488  0.010121  0.002113  0.002587  0.001623  0.004179   
18774  0.000014  0.000017  0.000025  0.000010  0.000011  0.000005  0.000040   
18775  0.000710  0.000583  0.002334  0.000714  0.001599  0.000291  0.001587   
18776  0.004456  0.004077  0.021462  0.006225  0.008126  0.003139  0.010010   

                7         8         9         10   

In [10]:
dat.to_csv('normalised.csv')

In [ ]:
pip install gseapy

     |████████████████████████████████| 526 kB 5.2 MB/s 
     |████████████████████████████████| 1.0 MB 35.4 MB/s 
     |████████████████████████████████| 47 kB 4.0 MB/s 
     |████████████████████████████████| 47 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
     |████████████████████████████████| 5.8 MB 35.5 MB/s 
     |████████████████████████████████| 251 kB 46.1 MB/s 
     |████████████████████████████████| 127 kB 46.5 MB/s 
  Created wheel for bioservices: filename=bioservices-1.8.4-py3-none-any.whl size=231822 sha256=db668a42a4e151d78022c51755cb760987a879d6b7b0fa20dde6d6478f297256
  Stored in directory: /root/.cache/pip/wheels/68/df/3c/0ed23eeeafac858b6d4d3ec54d90d58e9292628b50d43bcd0f
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64232 sha256=d1cb47da73fedf6dce0f8bdb7ca333784a2ff15e44a957291887e75e473e9c66
  Stored in directory: /root/.cache/pip/wheels/82/ab/83/fdfc4017ea44a585b6754752cc5f63f2d0d63fcc1317e7174b
Successfully 

In [1]:
from sklearn.manifold import TSNE

In [12]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(dat) 

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 550 samples in 0.008s...
[t-SNE] Computed neighbors for 550 samples in 0.381s...
[t-SNE] Computed conditional probabilities for sample 550 / 550
[t-SNE] Mean sigma: 4.257687
[t-SNE] KL divergence after 250 iterations with early exaggeration: 89.495689
[t-SNE] KL divergence after 1000 iterations: 1.228872


In [5]:
cd

/root
